# Optimisation 

## Logistics

In this work we are going to use Pulp for supply chain and logistics optimisation.

In [1]:
import os 
import pandas as pd 
%matplotlib inline 
import matplotlib 
import matplotlib.pyplot as plt 
import numpy as np 
from pulp import *
from itertools import product

Let's imagine we have piano company, in fact, we just deliver piano, everyday we recieve some orders, we have parnters in three different regions and we are able to cooperate with each of them separately (one partner per day). We should develop an alorithm which let us minimize our costs. Every piano can be delivered separetel, because it is fragile.

Let's initialize our markets and possible hubs and warehouses

In [2]:
markets = ['Paris', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne'] # where we deliver
warehouses = ['Val-d Oise', 'Yvelines','Essonne'] # where our suppliers are
import numpy as np
import random
costs_dict = [(m,w) for m in markets for w in warehouses]
x = np.array([random.random() for i in range(12)])*1000
costs_dict = dict(zip(costs_dict,x))
# I initilize it with random numbers, but, usually we can use real data from experience of the company
costs = pd.DataFrame(columns=markets,index=warehouses)

In [3]:
demand = np.array([random.randrange(10) for i in range(4)]) 
# similarly demand is random, but integer, because we can deliver only the whole piano
demand = zip(demand,markets)
demand = pd.DataFrame(demand)
demand.columns = ['demand','market']
demand = demand.set_index('market')
demand

,demand
market,
Paris,9
Hauts-de-Seine,6
Seine-Saint-Denis,4
Val-de-Marne,7


In [4]:
for w in warehouses:
    for m in markets:
        costs.loc[(w,m)] = costs_dict[(m,w)]
costs

,Paris,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne
Val-d Oise,722.021,688.763,372.206,63.1353
Yvelines,270.946,880.917,159.856,662.928
Essonne,523.858,32.3367,292.584,621.675


In [5]:
model = LpProblem("Choose_the_warehouses", LpMinimize)
var_dict = LpVariable.dicts('stock-loc', 
                            [warehouse for warehouse in warehouses], 
                            lowBound=0, cat="Binary")
model += lpSum([var_dict[warehouse]*costs.loc[warehouse, market]*demand.loc[market, 'demand'] for warehouse in warehouses for market in markets])

We are allowed to open only one warehouse 

In [6]:
model += var_dict['Val-d Oise']+var_dict['Yvelines']+var_dict['Essonne'] == 1

In [7]:
model.solve()

1

In [8]:
# Print status
print(LpStatus[model.status])

# Print variables
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Total Cost = ", value(model.objective))

Optimal
stock_loc_Essonne = 1.0
stock_loc_Val_d_Oise = 0.0
stock_loc_Yvelines = 0.0
Total Cost =  10430.807130988685


## Shawarma production 

We are schawarma producers at the railway station hub, who are specialized on two specific recipes: Awarma and Schawarma. The traffic is really intensive, and our production capacity is not high enough to fulfill the whole demand. Price Schawarma is 11 euro, price of Awarma is equal to 15 euro. Ingredients of Schawarma are 120 grams vegetables, 1 naan bread, 100 grams of meat. Ingredients of awarma are 200 grams of meat, 1 naan bread, 50 grams of houmous. We have three suppliers who send the products only in batches
Vegetable and Houmous supplier (one kilogram of vegetable and one kilogram of houmous in batch)
Meat supplier (one kilogram  of meat per batch)
Naan supplier (100 naan per batch)
price of meat=50 euro/kg, vegetables = 35 euro/kg naan = 5 cents, houmous = 28 euro/kg
Everytging we produced we are able to sell, however we cannot produce more than 1000 shawarmas/awarma per day, so what would be the appropriate production strategy?

In [9]:
s_price = 11
a_price = 15
# price per batch
meat_price = 50
vegetables_price = 35
houmous_price = 28
naan_price = 5

In [10]:
model = LpProblem("Maximize Profits", LpMaximize)
awarma = LpVariable('awarma', lowBound=0,upBound=None, cat = 'Integer')
schawarma = LpVariable('schawarma', lowBound=0,upBound=None, cat = 'Integer')
ingredients = ['meat','vegetables','houmous', 'naan']
costs_ing = LpVariable.dicts('batched_ordered', ingredients, lowBound = 0, cat = 'Integer')

C:\Users\eldar\Anaconda3\lib\site-packages\pulp\pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
costs_ing

{'meat': batched_ordered_meat,
 'vegetables': batched_ordered_vegetables,
 'houmous': batched_ordered_houmous,
 'naan': batched_ordered_naan}

In [12]:
model += lpSum([awarma*a_price+
                schawarma*s_price-
                costs_ing['meat']*meat_price-
                costs_ing['vegetables']*vegetables_price-
                costs_ing['houmous']*houmous_price-
                costs_ing['naan']*naan_price])

In [13]:
model += awarma+schawarma <= 1000
model += 0.2*awarma+0.1*schawarma <= costs_ing['meat']
model += 0.12*schawarma <= costs_ing['vegetables']
model += 0.05*awarma <= costs_ing['houmous']
model += lpSum([awarma+schawarma])*0.01 <= costs_ing['naan']
# we have one supplier for houmous and vegetables, we can buy this products only together 
model += costs_ing['vegetables'] == costs_ing['houmous']

In [14]:
model.solve()
# Print status
print(LpStatus[model.status])

# Print variables
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Total profit = ", value(model.objective))

Optimal
awarma = 700.0
batched_ordered_houmous = 35.0
batched_ordered_meat = 169.0
batched_ordered_naan = 10.0
batched_ordered_vegetables = 35.0
schawarma = 290.0
Total profit =  2985.0


Here is our solution, requested orders and the profit, which is maximized